<a href="https://colab.research.google.com/github/jinyang628/chatbotNLP/blob/main/chatBotNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torch.utils.data import Dataset
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

In [ ]:
class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r"))

        self.text = []
        for i in self.data:
            for j in i['dialog']:
                self.text.append(j['text'])

        for idx, i in enumerate(self.text):
            try:
                self.text[idx] = "<startofstring> "+i+" <bot>: "+self.text[idx+1]+" <endofstring>"
            except:
                break

        self.text = self.text[:5000]
        
        print(self.text[0])

        self.text_encoded = tokenizer(self.text,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.text_encoded['input_ids']
        self.attention_mask = self.text_encoded['attention_mask']

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
def train(chatData, model, optim):

    epochs = 2

    for i in tqdm.tqdm(range(epochs)):
        for text, attnMask in chatData:
            text = text.to(device)
            attnMask = attnMask.to(device)
            optim.zero_grad()
            loss = model(text, attention_mask=attnMask, labels=text).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state.pt")
        print(infer("hello how are you"))

def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    text = inp["input_ids"].to(device)
    attnMask = inp["attention_mask"].to(device)
    output = model.generate(text, attention_mask=attnMask )
    output = tokenizer.decode(output[0])
    return output


device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

chatData = ChatData("./chat_data.json", tokenizer)
chatData =  DataLoader(chatData, batch_size=64)

model.train()

optim = Adam(model.parameters(), lr=1e-3)

# Train the model
train(chatData, model, optim)

# Run the model
print("infer from model : ")
while True:
  inp = input()
  print(infer(inp))

<startofstring> I love iphone! i just bought new iphone! <bot>: Thats good for you, i'm not very into new tech <endofstring>
training .... 


  0%|          | 0/2 [00:11<?, ?it/s]


KeyboardInterrupt: ignored